In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory



llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit = 20,
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter3.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

def load_memory (_):
    return memory.load_memory_variables({})['history']

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know. don't make it up : \n\n {context}"),
        MessagesPlaceholder(variable_name='history'),
        ("human", "{question}"),
    ]
)

chain = {"context" : retriever, "question" : RunnablePassthrough(), "history": load_memory} | prompt | llm

chain.invoke("Is Aaronson guilty?")

def invoke_chain(question):
    results = chain.invoke(question)
    print(results)
    memory.save_context({"input":question},{"output":results.content})

Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 1444, which is longer than the specified 600
Created a chunk of size 1251, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 1493, which is longer than the specified 600
Created a chunk of size 819, which is longer than the specified 600
Created a chunk of size 1458, which is longer than the specified 600
Created a chunk of size 1411, which is longer than the specified 600
Created a chunk of size 742, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 703, which is longer than the specified 600
Created a chunk of size 1137, which is lon

In [2]:
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia")

content='Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'
content='He wrote: FREEDOM IS SLAVERY, TWO AND TWO MAKE FIVE, GOD IS POWER.'
content='Julia is a character who is important to the protagonist and plays a significant role in the story.'


In [3]:
load_memory({})

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content='Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='He wrote: FREEDOM IS SLAVERY, TWO AND TWO MAKE FIVE, GOD IS POWER.'),
 HumanMessage(content='Who is Julia'),
 AIMessage(content='Julia is a character who is important to the protagonist and plays a significant role in the story.')]